Test file to verify that LogisticBART and update_fit work together without bugs.

This test creates a simple classification scenario, fits a LogisticBART model,
then uses update_fit to add incremental data to ensure the incremental training
functionality operates correctly.

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from bart_playground import LogisticBART, DataGenerator

# %% functions

def test_logistic_bart_basic_functionality():
    """Test basic LogisticBART functionality before testing update_fit."""
    print("Testing basic LogisticBART functionality...")
    
    # Generate simple binary classification data
    X, y = make_classification(
        n_samples=100, 
        n_features=3, 
        n_redundant=0, 
        n_informative=3,
        n_clusters_per_class=1,
        n_classes=2,
        random_state=42
    )
    
    # Create LogisticBART model with small parameters for fast testing
    model = LogisticBART(
        ndpost=10,  # Small number for fast testing
        nskip=5,
        n_trees=5,
        random_state=42
    )
    
    # Fit the model
    model.fit(X, y, quietly=True)
    
    # Basic checks
    assert model.is_fitted
    assert len(model.trace) == 10  # ndpost samples
    assert model.sampler.n_categories == 2  # Binary classification
    
    # Test predictions
    predictions = model.predict(X)
    assert predictions.shape == (100,)
    assert np.all(np.isin(predictions, [0, 1]))  # Should be 0 or 1
    
    # Test probabilities
    probabilities = model.predict_proba(X)
    assert probabilities.shape == (100, 2)
    assert np.allclose(probabilities.sum(axis=1), 1.0)  # Should sum to 1
    
    print("✅ Basic LogisticBART functionality test passed!")


def test_logistic_bart_multiclass_functionality():
    """Test LogisticBART with multiclass classification."""
    print("Testing LogisticBART with multiclass classification...")
    
    # Generate multiclass classification data
    X, y = make_classification(
        n_samples=80,
        n_features=4,
        n_redundant=0,
        n_informative=4,
        n_clusters_per_class=1,
        n_classes=3,  # 3 classes
        random_state=42
    )
    
    # Create LogisticBART model
    model = LogisticBART(
        ndpost=8,
        nskip=4,
        n_trees=4,
        random_state=42
    )
    
    # Fit the model
    model.fit(X, y, quietly=True)
    
    # Basic checks
    assert model.is_fitted
    assert model.sampler.n_categories == 3  # Three classes
    
    # Test predictions
    predictions = model.predict(X)
    assert predictions.shape == (80,)
    assert np.all(np.isin(predictions, [0, 1, 2]))  # Should be 0, 1, or 2
    
    # Test probabilities
    probabilities = model.predict_proba(X)
    assert probabilities.shape == (80, 3)
    assert np.allclose(probabilities.sum(axis=1), 1.0)  # Should sum to 1
    
    print("✅ LogisticBART multiclass functionality test passed!")


def test_logistic_bart_update_fit_binary():
    """Test LogisticBART update_fit functionality with binary classification."""
    print("Testing LogisticBART update_fit with binary classification...")
    
    # Generate initial training data
    X_initial, y_initial = make_classification(
        n_samples=60,
        n_features=3,
        n_redundant=0,
        n_informative=3,
        n_clusters_per_class=1,
        n_classes=2,
        random_state=42
    )
    
    # Generate additional data for update_fit
    X_new, y_new = make_classification(
        n_samples=40,
        n_features=3,
        n_redundant=0,
        n_informative=3,
        n_clusters_per_class=1,
        n_classes=2,
        random_state=123  # Different seed for variety
    )
    
    # Create and fit initial model
    model = LogisticBART(
        ndpost=8,
        nskip=4,
        n_trees=5,
        random_state=42
    )
    
    model.fit(X_initial, y_initial, quietly=True)
    
    # Store initial state for comparison
    initial_trace_length = len(model.trace)
    initial_predictions = model.predict(X_initial)
    
    # Test update_fit
    model.update_fit(X_new, y_new, add_ndpost=6, add_nskip=3, quietly=True)
    
    # Verify update_fit worked
    assert model.is_fitted
    # assert len(model.trace) == initial_trace_length + 6  # Should have added add_ndpost samples
    assert model.data.n == 100  # Should have combined data (60 + 40)
    assert model.ndpost == 6  # Should be updated to add_ndpost
    
    # Test predictions on combined data
    X_combined = np.vstack([X_initial, X_new])
    y_combined = np.hstack([y_initial, y_new])
    
    predictions_combined = model.predict(X_combined)
    assert predictions_combined.shape == (100,)
    assert np.all(np.isin(predictions_combined, [0, 1]))
    
    probabilities_combined = model.predict_proba(X_combined)
    assert probabilities_combined.shape == (100, 2)
    assert np.allclose(probabilities_combined.sum(axis=1), 1.0)
    
    print("✅ LogisticBART update_fit binary classification test passed!")


def test_logistic_bart_update_fit_multiclass():
    """Test LogisticBART update_fit functionality with multiclass classification."""
    print("Testing LogisticBART update_fit with multiclass classification...")
    
    # Generate initial training data
    X_initial, y_initial = make_classification(
        n_samples=50,
        n_features=4,
        n_redundant=0,
        n_informative=4,
        n_clusters_per_class=1,
        n_classes=3,
        random_state=42
    )
    
    # Generate additional data for update_fit
    X_new, y_new = make_classification(
        n_samples=30,
        n_features=4,
        n_redundant=0,
        n_informative=4,
        n_clusters_per_class=1,
        n_classes=3,
        random_state=123
    )
    
    # Create and fit initial model
    model = LogisticBART(
        ndpost=6,
        nskip=3,
        n_trees=4,
        random_state=42
    )
    
    model.fit(X_initial, y_initial, quietly=True)
    
    # Store initial state
    initial_trace_length = len(model.trace)
    
    # Test update_fit
    model.update_fit(X_new, y_new, add_ndpost=4, add_nskip=2, quietly=True)
    
    # Verify update_fit worked
    assert model.is_fitted
    # assert len(model.trace) == initial_trace_length + 4  # Should have added add_ndpost samples
    assert model.data.n == 80  # Should have combined data (50 + 30)
    assert model.sampler.n_categories == 3  # Should still be 3 classes
    
    # Test predictions on combined data
    X_combined = np.vstack([X_initial, X_new])
    
    predictions_combined = model.predict(X_combined)
    assert predictions_combined.shape == (80,)
    assert np.all(np.isin(predictions_combined, [0, 1, 2]))
    
    probabilities_combined = model.predict_proba(X_combined)
    assert probabilities_combined.shape == (80, 3)
    assert np.allclose(probabilities_combined.sum(axis=1), 1.0)
    
    print("✅ LogisticBART update_fit multiclass classification test passed!")


def test_logistic_bart_trace_structure_after_update():
    """Test that the trace structure is correct after update_fit."""
    print("Testing LogisticBART trace structure after update_fit...")
    
    # Generate simple data
    X_initial, y_initial = make_classification(
        n_samples=40,
        n_features=2,
        n_redundant=0,
        n_informative=2,
        n_clusters_per_class=1,
        n_classes=2,
        random_state=42
    )
    
    X_new, y_new = make_classification(
        n_samples=20,
        n_features=2,
        n_redundant=0,
        n_informative=2,
        n_clusters_per_class=1,
        n_classes=2,
        random_state=123
    )
    
    # Create model
    model = LogisticBART(
        ndpost=5,
        nskip=2,
        n_trees=3,
        random_state=42
    )
    
    model.fit(X_initial, y_initial, quietly=True)
    
    # Check initial trace structure
    assert len(model.trace) == 5
    for params_list in model.trace:
        assert isinstance(params_list, list)
        assert len(params_list) == 2  # Binary classification
        for params in params_list:
            assert hasattr(params, 'trees')
            assert hasattr(params, 'global_params')
            assert len(params.trees) == 3  # n_trees
    
    # Update with new data
    model.update_fit(X_new, y_new, add_ndpost=3, add_nskip=1, quietly=True)
    
    # Check updated trace structure
    # assert len(model.trace) == 5 + 3  # original + add_ndpost
    for params_list in model.trace:
        assert isinstance(params_list, list)
        assert len(params_list) == 2  # Still binary classification
        for params in params_list:
            assert hasattr(params, 'trees')
            assert hasattr(params, 'global_params')
            assert len(params.trees) == 3  # Still n_trees
    
    print("✅ LogisticBART trace structure test passed!")


def test_logistic_bart_error_handling():
    """Test error handling in LogisticBART update_fit."""
    print("Testing LogisticBART error handling...")
    
    # Test update_fit on unfitted model
    model = LogisticBART(ndpost=5, nskip=2, n_trees=3, random_state=42)
    
    X_new, y_new = make_classification(
        n_samples=30,
        n_features=4,
        n_classes=2,
        random_state=42
    )
    
    # This should work - it should fall back to regular fit
    model.update_fit(X_new, y_new, quietly=True)
    assert model.is_fitted
    
    print("✅ LogisticBART error handling test passed!")


def test_data_generator_with_logistic_bart():
    """Test LogisticBART with DataGenerator classification data."""
    print("Testing LogisticBART with DataGenerator...")
    
    # Use DataGenerator for more realistic test data
    generator = DataGenerator(n_samples=80, n_features=3, noise=0.1, random_seed=42)
    X, y_continuous = generator.generate(scenario="piecewise_linear")
    
    # Convert to binary classification by thresholding
    y = (y_continuous > np.median(y_continuous)).astype(int)
    
    # Split data for update_fit test
    X_initial, X_new, y_initial, y_new = train_test_split(
        X, y, test_size=0.3, random_state=42
    )
    
    # Create and fit model
    model = LogisticBART(
        ndpost=6,
        nskip=3,
        n_trees=4,
        random_state=42
    )
    
    model.fit(X_initial, y_initial, quietly=True)
    
    # Test initial predictions
    pred_initial = model.predict(X_initial)
    assert pred_initial.shape == (len(X_initial),)
    
    # Update with new data
    model.update_fit(X_new, y_new, add_ndpost=4, add_nskip=2, quietly=True)
    
    # Test final predictions
    X_combined = np.vstack([X_initial, X_new])
    pred_combined = model.predict(X_combined)
    assert pred_combined.shape == (len(X_combined),)
    assert np.all(np.isin(pred_combined, [0, 1]))
    
    print("✅ LogisticBART with DataGenerator test passed!")
    
    assert np.mean(np.abs(pred_combined[0:X_initial.shape[0]] - pred_initial)) > 0.0
    assert np.mean(np.abs(pred_combined[0:X_initial.shape[0]] - pred_initial)) < 0.1
    print("✅ LogisticBART predictions consistency test passed!")

In [2]:
 
# %% main
print("Starting LogisticBART update_fit integration tests...\n")

try:
    test_logistic_bart_basic_functionality()
    print()
    
    test_logistic_bart_multiclass_functionality()
    print()
    
    test_logistic_bart_update_fit_binary()
    print()
    
    test_logistic_bart_update_fit_multiclass()
    print()
    
    test_logistic_bart_trace_structure_after_update()
    print()
    
    test_logistic_bart_error_handling()
    print()
    
    test_data_generator_with_logistic_bart()
    print()
    
    print("🎉 All LogisticBART update_fit tests passed successfully!")
    print("\nSummary:")
    print("- Basic LogisticBART functionality: ✅")
    print("- Multiclass classification: ✅")
    print("- Binary update_fit: ✅")
    print("- Multiclass update_fit: ✅")
    print("- Trace structure consistency: ✅")
    print("- Error handling: ✅")
    print("- DataGenerator integration: ✅")
    print("\nLogisticBART's incremental training via update_fit works correctly!")
    
except Exception as e:
    print(f"❌ Test failed with error: {e}")


Starting LogisticBART update_fit integration tests...

Testing basic LogisticBART functionality...
✅ Basic LogisticBART functionality test passed!

Testing LogisticBART with multiclass classification...
✅ LogisticBART multiclass functionality test passed!

Testing LogisticBART update_fit with binary classification...
✅ LogisticBART update_fit binary classification test passed!

Testing LogisticBART update_fit with multiclass classification...
✅ LogisticBART update_fit multiclass classification test passed!

Testing LogisticBART trace structure after update_fit...
✅ LogisticBART trace structure test passed!

Testing LogisticBART error handling...
✅ LogisticBART error handling test passed!

Testing LogisticBART with DataGenerator...
✅ LogisticBART with DataGenerator test passed!
✅ LogisticBART predictions consistency test passed!

🎉 All LogisticBART update_fit tests passed successfully!

Summary:
- Basic LogisticBART functionality: ✅
- Multiclass classification: ✅
- Binary update_fit: ✅
